In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Define the dataset class for test data
class TestAngleDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.copy()
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
            
        # Return image, region, and filename
        region = torch.tensor(row['region_encoded'], dtype=torch.long)
        return image, region, row['filename']

# Define the model architecture (same as in training)
class ImprovedAngleModel(nn.Module):
    def __init__(self, num_regions, backbone_name='efficientnet_b0'):
        super(ImprovedAngleModel, self).__init__()
        
        # Choose backbone based on parameter
        if backbone_name == 'resnet50':
            self.backbone = models.resnet50(pretrained=True)
            self.backbone.fc = nn.Identity()  # Remove final FC layer
            self.feature_dim = 2048
        elif backbone_name == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(pretrained=True)
            self.backbone.classifier = nn.Identity()  # Remove classifier
            self.feature_dim = 1280
        elif backbone_name == 'convnext_small':
            self.backbone = models.convnext_small(pretrained=True)
            self.backbone.classifier[2] = nn.Identity()  # Remove classifier
            self.feature_dim = 768
        else:  # Default to ResNet18
            self.backbone = models.resnet18(pretrained=True)
            self.backbone.fc = nn.Identity()  # Remove final FC layer
            self.feature_dim = 512
        
        # Region embedding
        self.region_embedding = nn.Embedding(num_regions, 128)
        
        # Attention mechanism for feature fusion
        self.attention = nn.Sequential(
            nn.Linear(self.feature_dim + 128, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
        
        # Final layers with multiple branches
        self.shared_fc = nn.Sequential(
            nn.Linear(self.feature_dim + 128, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        self.angle_fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 2)  # Output sin(θ), cos(θ)
        )
    
    def forward(self, x, regions):
        # Extract image features
        img_features = self.backbone(x)
        
        # Get region embeddings
        region_features = self.region_embedding(regions)
        
        # Concatenate features
        combined_features = torch.cat([img_features, region_features], dim=1)
        
        # Apply attention mechanism
        attention_weights = torch.sigmoid(self.attention(combined_features))
        attended_features = combined_features * attention_weights
        
        # Shared layers
        shared_features = self.shared_fc(attended_features)
        
        # Angle prediction branch
        angle_out = self.angle_fc(shared_features)
        
        return angle_out

# Function to get test transforms
def get_test_transform(img_size=288, crop_size=256):
    return transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def main():
    # Set seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Path configurations
    MODEL_PATH = '/kaggle/input/best_model/pytorch/default/1/best_model_angle.pth'  # Path to the saved model
    TEST_IMG_DIR = '/kaggle/input/images-test/images_test'  # Path to test images
    REGIONS_PATH = '/kaggle/input/predicted-regions-test/predicted_regions_test.csv'  # Path to predicted regions
    
    # Get test transform
    test_transform = get_test_transform(img_size=288, crop_size=256)
    
    # Load test data
    test_df = pd.read_csv(REGIONS_PATH)
    
    # Load region encodings from training data to ensure consistency
    # The simplest approach is to use the same regions as in training
    # For this example, we'll re-encode regions in this script
    # In practice, you should save the LabelEncoder during training
    region_encoder = LabelEncoder()
    
    # We'll assume regions from the test data might match what we had during training
    # If you have the original encoder or region mapping, use that instead
    test_df['region_encoded'] = region_encoder.fit_transform(test_df['Region_ID'])
    
    # Get number of unique regions
    NUM_REGIONS = len(region_encoder.classes_)
    print(f"Number of unique regions in test data: {NUM_REGIONS}")
    
    # Create test dataset
    test_dataset = TestAngleDataset(test_df, TEST_IMG_DIR, test_transform)
    
    # Create test dataloader
    test_loader = DataLoader(
        test_dataset, 
        batch_size=32, 
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    # Initialize model with the same architecture
    backbone_name = 'efficientnet_b0'  # Make sure this matches what was used in training
    model = ImprovedAngleModel(NUM_REGIONS, backbone_name=backbone_name).to(device)
    
    # Load weights
    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        print(f"Successfully loaded model from {MODEL_PATH}")
    except Exception as e:
        print(f"Error loading model: {e}")
        return
    
    # Set model to evaluation mode
    model.eval()
    
    # Generate predictions
    filenames = []
    predictions = []
    
    with torch.no_grad():
        for images, regions, batch_filenames in tqdm(test_loader, desc='Generating predictions'):
            images = images.to(device)
            regions = regions.to(device)
            
            outputs = model(images, regions)
            outputs = F.normalize(outputs, dim=1)
            
            # Convert sin(θ), cos(θ) predictions back to angles
            sin_preds = outputs[:, 0]
            cos_preds = outputs[:, 1]
            angle_preds = (torch.atan2(sin_preds, cos_preds) * 180 / np.pi) % 360
            
            filenames.extend(batch_filenames)
            predictions.extend(angle_preds.cpu().numpy())
    
    # Create results dataframe
    results_df = pd.DataFrame({
        'filename': filenames,
        'angle': predictions
    })
    
    # Sort by filename to ensure consistent ordering
    results_df = results_df.sort_values(by='filename')
    
    # Create submission file with id and angle columns
    submission_df = pd.DataFrame({
        'id': range(len(results_df)),
        'angle': results_df['angle'].values
    })
    
    # Save submission file
    submission_df.to_csv('2022101034_1.csv', index=False)
    print(f"Successfully saved predictions to 2022101034_1.csv")
    print(f"Total predictions: {len(submission_df)}")

if __name__ == "__main__":
    main()

Using device: cuda
Number of unique regions in test data: 15


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 119MB/s] 
/tmp/ipykernel_19/71489937.py:170: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses 

Successfully loaded model from /kaggle/input/best_model/pytorch/default/1/best_model_angle.pth


Generating predictions: 100%|██████████| 12/12 [00:03<00:00,  3.41it/s]

Successfully saved predictions to 2022101034_1.csv
Total predictions: 369
